# TMDB API calls

- Michael Vincent
- 9/20

## Imports

In [1]:
# Imports
import numpy as np
import pandas as pd
import os, time, json
import tmdbsimple as tmdb
from tqdm import tqdm_notebook

## File creation and API credentials

In [2]:
# Make sure the Data folder exists
folder = 'Data/'
os.makedirs(folder, exist_ok = True)
os.listdir(folder)

['title_basics.csv.gz',
 'title_akas.csv.gz',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2001.csv.gz']

In [3]:
# Get the API credentials
with open('/home/michael/.secret/tmdb_api.json') as f:
    login = json.load(f)

tmdb.API_KEY = login['api-key']

## Helper functions

In [4]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-
    python/"""
    
    with open(filename, 'r+') as file:
        # Load the existing data into a dictionary
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
            
        # Set the file's current position at offset
        file.seek(0)
        
        # Convert back to JSON
        json.dump(file_data, file)

In [5]:
def get_movie_with_rating(movie_id):
    """Adapted from: https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # Save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    
    # Loop through the countries in releases
    for c in releases['countries']:
        # Get the movies made in the US
        if (c['iso_3166_1'] == 'US'):
            # Save a "certification" key in info with the certification
            info['certification']= c['certification']
    return info

In [6]:
# Test the get_movie_with_rating function
display(get_movie_with_rating('tt0848228'))
display(get_movie_with_rating('tt0332280'))

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 230.762,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 70.353,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

> We see that our function is working correctly

## Load in the 'basics' data frame from part 1

In [7]:
basics = pd.read_csv('Data/title_basics.csv.gz', compression = 'gzip')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


## Make the API calls on TMDB

In [8]:
# Make a list of years to get certifications for
years_to_get = [2000, 2001]
# Make an empty list to store errors
errors = []

In [9]:
# Loop through years_to_get and basics
for year in tqdm_notebook(years_to_get, desc = 'YEARS', position = 0):
    
    # Define the JSON file to store the relults for 'year'
    json_file = f'{folder}tmdb_api_results_{year}.json'
    
    # Check if the file exisits
    if os.path.isfile(json_file):
        pass
    else: 
        with open(json_file, 'w') as f:
            # Save an emptf dict with key 'imdb_id'
            json.dump([{'imdb_id': 0}], f)
            
    # Make a data frame filtered by 'year'
    df = basics.loc[basics['startYear'] == year].copy()
    # Make a list of movie ids
    movie_ids = df['tconst'].copy()
    
    # Get the previous results
    previous_df = pd.read_json(json_file)
    # Filter out movies already in our data
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # Get the results for movie_ids_to_get
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc = f'Movies from {year}',
                                  position = 1,
                                  leave = True):
        
        try:
            # Get the data for the current movie
            temp = get_movie_with_rating(movie_id)
            # Append the results to the json file
            write_json(temp, json_file)
            # Pause to avoid overwhelming the server
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
    
    
    # Save the results 
    final_year_df = pd.read_json(json_file)
    final_year_df.to_csv(f'{folder}final_tmdb_data_{year}.csv.gz',
                         compression = 'gzip',
                         index = False)
# Display the number of errors
print(f'- Total Errors: {len(errors)}')

/tmp/ipykernel_22433/2587199376.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for year in tqdm_notebook(years_to_get, desc = 'YEARS', position = 0):


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_22433/2587199376.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie_id in tqdm_notebook(movie_ids_to_get,


Movies from 2000:   0%|          | 0/205 [00:00<?, ?it/s]

/tmp/ipykernel_22433/2587199376.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for movie_id in tqdm_notebook(movie_ids_to_get,


Movies from 2001:   0%|          | 0/241 [00:00<?, ?it/s]

- Total Errors: 446
